In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
load_dotenv()
import os 
os.environ['GOOGLE_API_KEY']=os.getenv("GOOGLE_API_KEY")
model_name=os.getenv("MODEL")

c:\Users\AkshayKumarBM\Documents\multimodel_extration\mme\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
Model_vision=ChatGoogleGenerativeAI(model=model_name,)

In [21]:
import base64

# Read local file using regular file operations
image_path = r"C:\Users\AkshayKumarBM\Documents\multimodel_extration\data\raw\ews-letter-1.jpg"

def GetImageDes(image_path, Model):

    with open(image_path, 'rb') as image_file:
        image_data = base64.b64encode(image_file.read()).decode("utf-8")

    message = HumanMessage(
        content=[
            {"type": "text", "text": "Act as Image Analyst. Analyze and extract insights from images. An expert in visual content interpretation and text Extraction with years of experience in image analysis "},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
            },
        ],
    )
    response = Model.invoke([message])
    # print(response.content)
    return response.content

In [16]:
response.content

'Here are the insights from the image:\n\n*   **Issuing Authority:** The letter is issued by the Commissionerate of Revenue Administration and Disaster Management.\n*   **Sender:** Dr. J. Radhakrishnan, I.A.S, Principal Secretary / Commissioner of Revenue Administration, Chennai.\n*   **Recipients:** All District Collectors and The Commissioner, Tamil Nadu e-Governance Agency, Chennai.\n*   **Subject:** Concerns income and asset certificates for the Economic Weaker Section, specifically regarding revised instructions.\n*   **Letter Number and Date:** Letter No. RA 5 (3) / 40521/2017, dated 04.06.2020.\n*   **References:** Cites previous circulars and government letters related to the subject.\n*   **Key Action:** Instructs Tahsildars not to issue income and asset certificates for the Economic Weaker Section either through online or manually.\n*   **Withdrawal:** The circular and letter issued by this office in reference vide 1 and 3 is hereby withdrawn.\n*   **Signatures:** Signed by J

In [ ]:
from pydantic import BaseModel , Field
from typing_extensions import List
def SuggestQuestion(ImageDest,Model_vision):

    class Question(BaseModel):
        question : str = Field(None,description="Suggested questions based on given data")

    class SuggestQue(BaseModel):
        questions : List[Question]

    Query_llm=Model_vision.with_structured_output(SuggestQue)
    result=Query_llm.invoke(ImageDest)
    return result


In [52]:
result_img=GetImageDes(image_path, Model_vision)
result_img

'Here are the insights from the image:\n\n*   **Header**: The document originates from the "Commissionerate of Revenue Administration and Disaster Management."\n*   **Sender**: The letter is from Dr. J. Radhakrishnan, I.A.S, who holds the position of Principal Secretary / Commissioner of Revenue Administration in Chepauk, Chennai.\n*   **Recipient**: The letter is addressed to (1) All District Collectors and (2) The Commissioner, Tamil Nadu e-Governance Agency, Chennai.\n*   **Subject**: The subject of the letter concerns "Income and asset certificate for Economic Weaker Section - revised instructions - Reg."\n*   **References**: The letter references previous circulars and government letters.\n*   **Content**: The main point of the letter is to inform that the circular and letter issued by this office in reference vide 1 and 3 is hereby withdrawn. It instructs Tahsildars not to issue the income and asset certificate for the Economic Weaker Section either through online or manually.\n*

In [53]:
result=SuggestQuestion(result_img)

In [54]:
result.questions

[Question(question='What is the letter about?'),
 Question(question='Who is the letter addressed to?'),
 Question(question='Who is the letter from?'),
 Question(question='When was the letter written?')]

In [55]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate(
    [   SystemMessage(content="Act as a Teacher, Based on the information you know answer the user Questions"),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("Questions : {Questions}"),
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

legacy_chain = LLMChain(
    llm=Model_vision,
    prompt=prompt,
    memory=memory,
)

legacy_result = legacy_chain.invoke({"Questions": "hi"})


In [56]:
memory.save_context(
    {"text": "Image Description is:"},  # Human message
    {"text": result_img},  # AI response
)


In [57]:
legacy_result = legacy_chain.invoke({"Questions": "What is the letter about"})

In [59]:
legacy_result['text']

"Okay, based on the image description, here's what the letter is about:\n\nThe letter is about **revised instructions regarding the Income and Asset Certificate for the Economically Weaker Section (EWS).**\n\nSpecifically, it informs the recipients (District Collectors and the Commissioner of the Tamil Nadu e-Governance Agency) that previous instructions regarding the issuance of this certificate are being withdrawn. The letter instructs Tahsildars to stop issuing the Income and Asset Certificate for the EWS, both online and manually."